# import libraries

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models  import Sequential
from tensorflow.keras.layers import Conv2D, Activation, Dropout, Dense, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam, Adadelta
import numpy as np
import tensorflow as tf
import keras
from keras.datasets import cifar10

# splitting data

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape)
image_size=32
x_train=np.array(x_train).reshape(-1, image_size ,image_size ,3)
x_test=np.array(x_test).reshape(-1, image_size ,image_size ,3)
#scale image =3 channel and all the data are scale
print(x_train.shape)
print(x_test.shape)
x_train,x_test=x_train/255.0 , x_test/255.0
print(x_train.shape[1:])

(50000, 32, 32, 3)
(50000, 32, 32, 3)
(10000, 32, 32, 3)
(32, 32, 3)


# AlexNet Model

In [3]:
# 5 Convolution layer and 3 fully connected 

AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape= x_train.shape[1:], kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

# 3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(10, activation='softmax', name='output'))

#Model Summary
AlexNet.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8, 8, 96)          34944     
                                                                 
 activation (Activation)     (None, 8, 8, 96)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 4, 96)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 256)         614656    
                                                                 
 activation_1 (Activation)   (None, 4, 4, 256)         0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2, 2, 256)        0         
 2D)                                                    

In [4]:
optimize = SGD(learning_rate=0.05)
AlexNet.compile(optimizer=optimize, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
AlexNet.fit(x_train, y_train, epochs=5 ,batch_size=20)
loss, accuracy= AlexNet.evaluate(x_test, y_test)
print( 'the Accuracy =' , accuracy*100)

Epoch 1/5
2500/2500 [==============================] - 337s 134ms/step - loss: 2.0716 - accuracy: 0.2050
Epoch 2/5
2500/2500 [==============================] - 339s 136ms/step - loss: 1.6527 - accuracy: 0.3883
Epoch 3/5
2500/2500 [==============================] - 366s 146ms/step - loss: 1.4321 - accuracy: 0.4828
Epoch 4/5
2500/2500 [==============================] - 376s 150ms/step - loss: 1.2934 - accuracy: 0.5373
Epoch 5/5
313/313 [==============================] - 6s 19ms/step - loss: 1.2109 - accuracy: 0.5762
the Accuracy = 95.6200008392334


# Prediction for AlexNet

In [5]:
y_pred=AlexNet.predict(x_test)
y_pre=np.argmax(y_pred,axis=1)
y_pred
     

313/313 [==============================] - 6s 20ms/step


array([[1.4927943e-02, 2.1470008e-02, 4.1298833e-02, ..., 3.3384856e-02,
        3.4973290e-02, 2.7065877e-02],
       [4.2398870e-02, 4.6362948e-02, 1.2835080e-04, ..., 5.5462671e-05,
        8.8961482e-01, 2.1169465e-02],
       [2.1539201e-01, 1.0836257e-02, 3.1462056e-03, ..., 9.2721323e-04,
        7.4340898e-01, 2.0134624e-02],
       ...,
       [3.3589124e-04, 6.8375584e-06, 9.2258073e-02, ..., 1.1019549e-02,
        4.7673690e-05, 3.8462571e-05],
       [3.8794920e-02, 8.2279164e-03, 2.1727961e-01, ..., 7.3652178e-02,
        1.1018097e-02, 9.7554130e-03],
       [3.4476838e-03, 8.1933336e-04, 2.2828050e-02, ..., 5.5747831e-01,
        3.4250107e-04, 3.2504816e-03]], dtype=float32)

# Classification_report for AlexNet

In [6]:
from sklearn.metrics import f1_score,confusion_matrix,recall_score,precision_score
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pre))
     

              precision    recall  f1-score   support

           0       0.66      0.67      0.66      1000
           1       0.73      0.67      0.70      1000
           2       0.58      0.31      0.40      1000
           3       0.35      0.41      0.38      1000
           4       0.45      0.56      0.50      1000
           5       0.55      0.40      0.47      1000
           6       0.54      0.77      0.63      1000
           7       0.68      0.69      0.68      1000
           8       0.60      0.80      0.68      1000
           9       0.78      0.50      0.61      1000

    accuracy                           0.58     10000
   macro avg       0.59      0.58      0.57     10000
weighted avg       0.59      0.58      0.57     10000



# Confusion_matrix for AlexNet

In [7]:
print(confusion_matrix(y_test,y_pre))

[[666   8  35  30  35   3  24  15 172  12]
 [ 49 668   1  30  12   1  27  19 115  78]
 [ 85  13 311 118 220  54 120  39  30  10]
 [ 21  12  33 405  88 157 194  44  34  12]
 [ 32   4  68  64 556  30 113 100  32   1]
 [  8   4  42 282  85 404  94  58  18   5]
 [  5  10  32  54 100  11 767   5  14   2]
 [ 21   6   8  75  98  68  25 686   8   5]
 [ 74  37   2  29  20   4  16   6 796  16]
 [ 52 153   4  65  23   2  45  44 109 503]]


# VGG-16 Model

In [8]:
# 13 Convolution layer and 2 fully connected 
model = Sequential()
model.add(Conv2D(input_shape= x_train.shape[1:],filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2), padding="same",strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2), padding="same",strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2), padding="same",strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2), padding="same",strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))

model.add(Dense(units=10, activation="softmax"))

In [9]:

optimize = SGD(learning_rate=0.05)
model.compile(optimizer=optimize, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5 ,batch_size=20)
loss, accuracy= model.evaluate(x_test, y_test)
print( 'the Accuracy =' , accuracy*100)

Epoch 1/5
2500/2500 [==============================] - 1464s 585ms/step - loss: 2.3033 - accuracy: 0.0970
Epoch 2/5
2500/2500 [==============================] - 2172s 869ms/step - loss: 2.2992 - accuracy: 0.1076
Epoch 3/5
2500/2500 [==============================] - 1467s 587ms/step - loss: 2.3033 - accuracy: 0.0996
Epoch 4/5
2500/2500 [==============================] - 1576s 630ms/step - loss: 2.3034 - accuracy: 0.0979
Epoch 5/5
313/313 [==============================] - 29s 91ms/step - loss: 2.3036 - accuracy: 0.1000
the Accuracy = 55.00000014901161


# Prediction for for VGG-16 Model

In [10]:
y_pred=model.predict(x_test)
y_pre=np.argmax(y_pred,axis=1)  #The numpy.argmax() function returns indices of the max element of the array
y_pred
     

313/313 [==============================] - 29s 91ms/step


array([[0.09945872, 0.09859838, 0.10106259, ..., 0.10025483, 0.10249443,
        0.09735675],
       [0.09946029, 0.09859869, 0.10106175, ..., 0.10025474, 0.1024956 ,
        0.09735764],
       [0.09946091, 0.09859914, 0.10106127, ..., 0.10025466, 0.10249671,
        0.09735827],
       ...,
       [0.09945676, 0.09859734, 0.10106382, ..., 0.10025502, 0.10249204,
        0.09735547],
       [0.09945954, 0.09859849, 0.10106213, ..., 0.10025473, 0.10249527,
        0.09735736],
       [0.09946018, 0.09859879, 0.10106179, ..., 0.10025465, 0.10249592,
        0.09735769]], dtype=float32)

# Classification_report for VGG-16 Model

In [11]:
from sklearn.metrics import f1_score,confusion_matrix,recall_score,precision_score
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pre))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1000
           1       0.00      0.00      0.00      1000
           2       0.00      0.00      0.00      1000
           3       0.00      0.00      0.00      1000
           4       0.10      1.00      0.18      1000
           5       0.00      0.00      0.00      1000
           6       0.00      0.00      0.00      1000
           7       0.00      0.00      0.00      1000
           8       0.00      0.00      0.00      1000
           9       0.00      0.00      0.00      1000

    accuracy                           0.10     10000
   macro avg       0.01      0.10      0.02     10000
weighted avg       0.01      0.10      0.02     10000



C:\Users\smart\Downloads\Programs\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\smart\Downloads\Programs\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\smart\Downloads\Programs\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Confusion_matrix for VGG-16 Model

In [12]:
print(confusion_matrix(y_test,y_pre))

[[   0    0    0    0 1000    0    0    0    0    0]
 [   0    0    0    0 1000    0    0    0    0    0]
 [   0    0    0    0 1000    0    0    0    0    0]
 [   0    0    0    0 1000    0    0    0    0    0]
 [   0    0    0    0 1000    0    0    0    0    0]
 [   0    0    0    0 1000    0    0    0    0    0]
 [   0    0    0    0 1000    0    0    0    0    0]
 [   0    0    0    0 1000    0    0    0    0    0]
 [   0    0    0    0 1000    0    0    0    0    0]
 [   0    0    0    0 1000    0    0    0    0    0]]
